In [ ]:
import os

os.environ['DJANGO_SETTINGS_MODULE'] = 'config.settings.devel'

import django

django.setup()

from organizations.models import Organization
from publications.models import Platform
from sushi.models import SushiCredentials
from core.models import User

platforms_to_keep = [
 'ACM Digital Library',
 'ACS',
 'BioOne',
 'CUP',
 'DeGruyter',
 'EbscoHost',
 'IEEE Xplore Digital Library',
 'IOPscience',
 'JSTOR',
 'Nature.com',
 'OUP',
 'ProQuest',
 'ProQuest Ebook Central',
 'RSC',
 'Sage',
 'ScienceDirect',
 'Scitation',
 'Scopus',
 'SpringerLink',
 'Taylor & Francis Online',
 'Taylor and Francis ebooks',
 'Web Of Science',
 'Wiley Online Library'
]

organizations_to_keep = [
    'National Library of Technology',
    'Library of the Czech Academy of Sciences',
    'Czech Technical University in Prague',
    'Czech University of Life Sciences Prague',
    'Masaryk University',
    'Veterinary Research Institute',
    'University of Chemistry and Technology,  Prague',
]

users_to_keep = [
    'beda.kosata@gmail.com',
]

[x.name for x in Organization.objects.all()]


In [ ]:
print(SushiCredentials.objects.all().delete())
print(Platform.objects.exclude(name__in=platforms_to_keep).delete())
# print(Organization.objects.exclude(name__in=organizations_to_keep).delete())
print(User.objects.exclude(email__in=users_to_keep).delete())

In [ ]:
from faker import Faker

fake = Faker()

for org in Organization.objects.all():
    name = fake.company()
    short_name = ''.join(l for l in name if l.isupper())
    org.url = fake.url()
    org.name_en = name
    org.name_cs = name
    org.short_name_cs = short_name
    org.short_name_en = short_name
    org.ext_id = fake.random_number()
    org.internal_id = f'{short_name}-{fake.random_number(digits=4)}'
    org.ico = fake.random_number(digits=8, fix_len=False)
    org.address = {}
    org.fte = fake.random_number(digits=3)
    
    
    org.save()

    

In [ ]:
from publications.models import Title
from django.db.models import Count

Title.objects.annotate(x=Count('platformtitle')).filter(x=0).delete()


In [ ]:
from logs.models import AccessLog, ImportBatch

print('accesslogs', AccessLog.objects.count())
print('batches', ImportBatch.objects.count())

ImportBatch.objects.annotate(x=Count('accesslog')).filter(x__gt=0).count()


In [ ]:
# prepocitat zajem
from logs.logic.materialized_interest import recompute_interest_by_batch

recompute_interest_by_batch(ImportBatch.objects.all())

In [ ]:
# smazat annotace, vytvořit nové fakové

from annotations.models import Annotation

Annotation.objects.all().delete()
for i in range(3):
    Annotation.objects.create(
        organization=fake.random_element(list(Organization.objects.all())+[None]),
        platform=fake.random_element(list(Platform.objects.all())+[None]),
        subject=fake.sentence(),
        message='\n'.join(fake.paragraphs(nb=2)),
        level=fake.random_element([x[0] for x in Annotation.LEVEL_CHOICES])
    )

In [ ]:
# smazat sushi task results
# smazat errors

from django_celery_results.models import TaskResult
from error_report.models import Error
print(TaskResult.objects.all().delete())
print(Error.objects.all().delete())


In [ ]:
# accesslogs - fake values

als = []
for al in AccessLog.objects.all():
    offset = fake.random_int(min=1, max=100)
    multi = fake.random_int(min=1, max=1000) / 30
    al.value = int(al.value * multi) + offset 
    als.append(als)
    
AccessLog.objects.bulk_update(als, fields=['value'])


In [ ]:
# smazat nebo nejak pozmenit payments

from cost.models import Payment

payments = []
for p in Payment.objects.all():
    offset = fake.random_int(min=1, max=100)
    multi = fake.random_int(min=1, max=100) / 100
    p.price = 10 * (int(p.price * multi) + offset) 
    payments.append(p)
    
Payment.objects.bulk_update(payments, fields=['price'])


In [ ]:
# vytvorit fake sushi credentials

